# FarmerGPT - Build Chat with PDF using lancedb

We utilized this PDF: Dataset/FARM5CROPS_farmergpt.pdf,
 which contains crop variety information for sugarcane, turmeric, bamboo, cashew nuts, and more.
 This is a sample project designed to demonstrate how to build an application using LanceDB and LangChain
The use case and prompts can be customized as needed to suit specific requirements

Import pacages

In [1]:
! pip install langchain lancedb
! pip install tantivy
! pip install -U langchain-openai langchain-community pypdf
! pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.8/34.8 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00


In [ ]:
# pass opeani key or use any LLM
import os

os.environ["OPENAI_API_KEY"] = "sk-proj-..."

In [8]:
# Download the sample pdf
!wget https://raw.githubusercontent.com/lancedb/vectordb-recipes/main/examples/RAG-On-PDF/Dataset/FARM5CROPS_farmergpt.pdf

--2025-07-29 16:07:18--  https://raw.githubusercontent.com/lancedb/vectordb-recipes/main/examples/RAG-On-PDF/Dataset/FARM5CROPS_farmergpt.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1398674 (1.3M) [application/octet-stream]
Saving to: ‘FARM5CROPS_farmergpt.pdf.1’

FARM5CROPS_farmergp 100%[===================>]   1.33M  --.-KB/s    in 0.05s   

2025-07-29 16:07:18 (28.8 MB/s) - ‘FARM5CROPS_farmergpt.pdf.1’ saved [1398674/1398674]



In [9]:
from lancedb.rerankers import LinearCombinationReranker
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import LanceDB
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ConversationBufferMemory


class QueryProcessor:
    def __init__(self, file_path, db_url="lancedb_temp", table_name="lancedb_indic"):
        """
        Initialize the QueryProcessor with the PDF file and set up the vector store.

        Parameters:
            file_path (str): Path to the PDF file.
            db_url (str): URI for the LanceDB vector store.
            table_name (str): Name of the table in LanceDB.
        """
        # Load and process the PDF document
        loader = PyPDFLoader(file_path)
        documents = loader.load()
        text_splitter = CharacterTextSplitter()
        self.documents = text_splitter.split_documents(documents)

        # Initialize embeddings and vector store
        embeddings = OpenAIEmbeddings()
        self.vector_store = LanceDB(
            uri=db_url, embedding=embeddings, table_name=table_name
        )

        # Add reranker
        self.reranker = LinearCombinationReranker(weight=0.3)
        self.docsearch = LanceDB.from_documents(
            self.documents, embeddings, reranker=self.reranker
        )

        print("Embedding stored in lancedb")
        # Initialize LLM and memory
        self.llm = ChatOpenAI(
            model_name="gpt-4o",
            temperature=0.01,
        )
        self.memory = ConversationBufferMemory(memory_key="chat_history")

    def generate_prompt_template(
        self, main_instructions, prompt_instructions, context_name, query
    ):
        """
        Generate a prompt template for LangChain LLM.

        Parameters:
            main_instructions (str): Main instructions for the LLM.
            prompt_instructions (str): Additional instructions for how to use the data.
            context_name (str): The name of the context (e.g., search results).
            query (str): The query from the user.

        Returns:
            PromptTemplate: The generated prompt template.
        """
        template = f"""{main_instructions}

        {prompt_instructions}

        {context_name}:
        {{context}}

        Previous Conversations:
        {{chat_history}}
        Human: {query}
        Chatbot:"""
        return PromptTemplate(
            template=template, input_variables=["context", "chat_history"]
        )

    def get_answer(self, query):
        """
        Process a query and return the answer based on the preloaded PDF.

        Parameters:
            query (str): The user's query.

        Returns:
            str: The answer to the query.
        """
        # Perform similarity search
        docs = self.docsearch.similarity_search_with_relevance_scores(query)

        # Generate a prompt
        prompt = self.generate_prompt_template(
            main_instructions="Act as a knowledgeable assistant. Answer the query comprehensively and concisely based on the provided content.",
            prompt_instructions=(
                "Focus on extracting the most relevant and accurate information from the context. "
                "Prioritize clarity, conciseness, and detail in your response. "
                "When summarizing, ensure key points are highlighted without losing important nuances. "
                "If the context is insufficient to fully address the query, acknowledge the limitation clearly."
            ),
            context_name="PDF Content",
            query=query,
        )

        # Create the LangChain pipeline
        chain = prompt | self.llm | StrOutputParser()

        # Invoke the chain and get the answer
        answer = chain.invoke({"context": docs, "chat_history": self.memory})
        return answer


# Initialize the QueryProcessor with the PDF file (done once)
file_path = "FARM5CROPS_farmergpt.pdf"
query_processor = QueryProcessor(file_path)

Embedding stored in lancedb


/tmp/ipython-input-9-1109352485.py:45: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  self.memory = ConversationBufferMemory(memory_key="chat_history")


In [10]:
query = "give me some sugarcane variety names?"
answer = query_processor.get_answer(query)
print("Answer:", answer)

Answer: Here are some sugarcane varieties from different states in India:

1. **Andhra Pradesh**:
   - Early varieties: Co.6907, 84A125, 81A99, 83A30, 85A261, 87A298, Co.8014, 86V96, 91V83.
   - Mid-late varieties: COA7607, CO8021, COT.8201, Co7805, COV92102 (83V15), 83V288.
   - Late varieties: Co.7219, CoR8001, 87A380, Co7706.

2. **Bihar**:
   - Bo 99, CoP 9301, CoSe 98231, CoS 8436, Cos 95255, Bo 102, Bo 91, Bo 110, CoP 9206, CoSe 95422, CoSe 92423, UP 9530.

3. **Gujarat**:
   - Co 86002, Co 86032, CoSi 95071, Co 86249, CoN 05072.

4. **Haryana**:
   - CoJ 64, CoS 8436, CoS 88230, CoS 767.

5. **Karnataka**:
   - Co 94012, CoC 671, Co 92020, Co 8014, Co 86032, Co 62175, Co 8371, Co 740, Co 8011.

6. **Maharashtra**:
   - CoC 671, Co 86032, Co 8011, Co 94012, CoM 265, Co 92005.

7. **Odisha**:
   - Co 62175, CoA 89085, Co 87A298, Co86V96.

8. **Punjab**:
   - CoJ 85, CoJ 88, CoS8436, CoH 119, Co89003.

9. **Tamil Nadu**:
   - Co 94012, Co 94010, CoC 24.

10. **Uttar Pradesh**:
    

In [11]:
# Reuse with another query
query2 = "What crops grow in dry regions?"
answer2 = query_processor.get_answer(query2)
print("Answer:", answer2)

Answer: Based on the provided content, sugarcane is a significant crop in India, grown in both tropical and subtropical regions. However, the document does not specifically address crops that grow in dry regions outside of sugarcane. It does mention that sugarcane is grown in areas with varying climates, including semi-arid regions, which are typically dry. In these areas, sugarcane cultivation is supported by irrigation and specific planting methods to manage water efficiently.

For a more comprehensive list of crops suitable for dry regions, additional information beyond the provided content would be necessary. Typically, crops like millet, sorghum, and certain legumes are known to be more drought-resistant and are often grown in dry regions.


# General template for your chat with pdf
##### change the promts as per requirements

In [ ]:
# gradio app

import gradio as gr
from lancedb.rerankers import LinearCombinationReranker
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import LanceDB
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ConversationBufferMemory


class QueryProcessor:
    def __init__(self, file_path, db_url="lancedb_temp", table_name="lancedb_indic"):
        loader = PyPDFLoader(file_path)
        documents = loader.load()
        text_splitter = CharacterTextSplitter()
        self.documents = text_splitter.split_documents(documents)

        embeddings = OpenAIEmbeddings()
        self.vector_store = LanceDB(
            uri=db_url, embedding=embeddings, table_name=table_name
        )

        self.reranker = LinearCombinationReranker(weight=0.3)
        self.docsearch = LanceDB.from_documents(
            self.documents, embeddings, reranker=self.reranker
        )

        self.llm = ChatOpenAI(
            model_name="gpt-4o",
            temperature=0.01,
        )
        self.memory = ConversationBufferMemory(memory_key="chat_history")

    def generate_prompt_template(
        self, main_instructions, prompt_instructions, context_name, query
    ):
        template = f"""{main_instructions}

        {prompt_instructions}

        {context_name}:
        {{context}}

        Previous Conversations:
        {{chat_history}}
        Human: {query}
        Chatbot:"""
        return PromptTemplate(
            template=template, input_variables=["context", "chat_history"]
        )

    def get_answer(self, query):
        docs = self.docsearch.similarity_search_with_relevance_scores(query)

        prompt = self.generate_prompt_template(
            main_instructions="Act as a knowledgeable assistant. Answer the query comprehensively and concisely based on the provided content.",
            prompt_instructions=(
                "Focus on extracting the most relevant and accurate information from the context. "
                "Prioritize clarity, conciseness, and detail in your response. "
                "When summarizing, ensure key points are highlighted without losing important nuances. "
                "If the context is insufficient to fully address the query, acknowledge the limitation clearly."
            ),
            context_name="Search Results",
            query=query,
        )

        chain = prompt | self.llm | StrOutputParser()

        answer = chain.invoke({"context": docs, "chat_history": self.memory})
        return answer


def initialize_processor(pdf_file):
    global query_processor
    query_processor = QueryProcessor(pdf_file.name)
    return "PDF successfully loaded and processed. You can now ask questions."


def query_processor_fn(question):
    global query_processor
    if query_processor is None:
        return "Please upload a PDF first."
    return query_processor.get_answer(question)


query_processor = None

# Define Gradio interface
with gr.Blocks() as app:
    gr.Markdown("# RAG On PDF - FarmersGPT")

    with gr.Row():
        pdf_upload = gr.File(label="Upload PDF", file_types=[".pdf"])
        pdf_status = gr.Textbox(label="Status", interactive=False)

    load_pdf_btn = gr.Button("Load PDF")

    with gr.Row():
        user_query = gr.Textbox(
            label="Ask a question", placeholder="Enter your question here..."
        )
        answer_box = gr.Textbox(label="Answer", interactive=False)

    ask_question_btn = gr.Button("Get Answer")

    load_pdf_btn.click(initialize_processor, inputs=[pdf_upload], outputs=[pdf_status])
    ask_question_btn.click(
        query_processor_fn, inputs=[user_query], outputs=[answer_box]
    )

app.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://82384a2b41c5d0935c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
